11.08.2022

# Find spark peaks / process sparks in annotations

Creo e testo una funzione per trovare i peaks nelle annotazioni degli sparks. Dopodiché la aggiungo alla Dataset class e i peaks saranno un attributo della classe e verranno usati per calcolare le metrics degli sparks.

Creo e testo anche una funzione per rimpicciolire gli sparks nelle annotazioni. Se funziona, la aggiungo alla Dataset class e creo un opzione da scegliere durante il training (se usare le raw annotations oppure quelle con gli sparks rimpiccioliti).

Quando ho finito e aggiunto le opzioni ai datasets, elimino questo script.

In [1]:
# autoreload is used to reload modules automatically before entering the
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# To import modules from parent directory in Jupyter Notebook
import sys

sys.path.append("..")

In [2]:
import napari

import numpy as np

from data.data_processing_tools import reduce_sparks_size, detect_spark_peaks
from utils.in_out_tools import load_movies_ids, load_annotations_ids
from utils.visualization_tools import get_discrete_cmap, get_labels_cmap

### load annotations and smooth movies

load movies and annotations from dataset folder --> same preprocessing as in the SparkDataset class

In [14]:
# sample_ids = ["01","02","03","04","05","06","07","08","09",
#             "10","11","12","13","14","15","16","17","18","19",
#             "20","21","22","23","24","25","27","28","29",
#             "30","32","33","34","35","36","38","39",
#             "40","41","42","43","44","45","46"
#            ]

# test_sample_ids = ["05","10","15","20","25","32","34","40","45"]
sample_ids = ["18"]

In [15]:
# directory containing dataset
base_path = "../data/sparks_dataset"
smooth_movies_path = "../data/raw_data_and_processing/smoothed_movies"

In [16]:
# load original movies
# data = list(load_movies_ids(data_folder=base_path,
#                            ids=sample_ids,
#                            #ids=test_sample_ids,
#                            names_available=True,
#                            movie_names="video"
#                           ).values())

# load smooth movies
data = list(
    load_movies_ids(
        data_folder=smooth_movies_path,
        ids=sample_ids,
        # ids=test_sample_ids,
        names_available=True,
        movie_names="smoothed_video",
    ).values()
)

# load class labels
annotations = list(
    load_annotations_ids(
        data_folder=base_path,
        ids=sample_ids,
        # ids=test_sample_ids,
        mask_names="class_label",
    ).values()
)

# load event labels
events = list(
    load_annotations_ids(
        data_folder=base_path,
        ids=sample_ids,
        # ids=test_sample_ids,
        mask_names="event_label",
    ).values()
)

### napari tools

In [17]:
cmap = get_discrete_cmap(name="gray", lut=16)

In [18]:
labels_cmap = get_labels_cmap()

### define function to reduce sparks size

the spark ROI that is removed will be the undefined region of the event

In [19]:
k = 75  # choose value k of percentile
sigma = 0  # use 2 with original movies, 0 with smooth movies

#### visualise result using napari

In [20]:
sample_id = 0
movie = data[sample_id]
class_mask = annotations[sample_id]
event_mask = events[sample_id]

spark_mask = np.where(class_mask == 1, event_mask, 0)

In [52]:
new_class_mask = reduce_sparks_size(
    movie=movie, class_mask=class_mask, event_mask=event_mask, sigma=sigma
)

[1516 1517 1518 1519]
[34 35]
[166 167 168 169]
[799 800 801]
[16]
[182 183]


In [55]:
viewer = napari.Viewer()
viewer.add_image(movie, name="movie", colormap=("colors", cmap))

viewer.add_labels(event_mask, name="events mask", opacity=0.5, visible=False)

viewer.add_labels(
    class_mask, name="classes mask", opacity=0.5, visible=False, color=labels_cmap
)

viewer.add_labels(spark_mask, name="sparks mask", opacity=0.6, visible=False)

<Labels layer 'sparks mask' at 0x2dfb9c55100>

In [56]:
viewer.add_labels(
    new_class_mask,
    name="new classes mask",
    opacity=0.6,
    visible=True,
    color=labels_cmap,
)

<Labels layer 'new classes mask' at 0x2dfa821e670>

#### compute peaks for all movies

In [90]:
idx = 0
for movie, class_mask, event_mask in zip(data, annotations, events):
    print(f"Processing {idx}th movie...")
    spark_mask = np.where(class_mask == 1, event_mask, 0)
    spark_loc = detect_spark_peaks(
        movie=movie,
        instances_mask=spark_mask,
        sigma=2,
        return_mask=False,
    )

    print("sparks locations:")
    print(spark_loc)
    idx = idx + 1

Processing 0th movie...
sparks locations:
[[196, 41, 303], [186, 49, 142], [58, 25, 108], [64, 15, 107], [107, 10, 118], [77, 41, 102], [70, 46, 114], [79, 16, 194], [283, 20, 336], [354, 45, 163], [95, 56, 143], [371, 13, 255], [258, 28, 326], [312, 11, 113], [196, 22, 198], [445, 49, 75], [326, 20, 233], [211, 37, 82], [406, 54, 170], [10, 55, 122], [454, 40, 297], [262, 32, 291], [93, 11, 112], [38, 47, 67], [309, 28, 86], [141, 22, 197], [425, 51, 109], [101, 21, 305], [48, 44, 163], [25, 44, 245], [352, 44, 177], [405, 11, 362], [415, 20, 337], [65, 13, 190], [21, 8, 381], [228, 55, 170], [289, 16, 338], [82, 20, 255], [415, 28, 326], [69, 39, 297], [42, 25, 150], [303, 21, 138], [351, 52, 156], [58, 6, 351], [402, 53, 265], [133, 47, 182], [208, 56, 142], [475, 3, 344], [220, 13, 339], [124, 26, 85], [219, 30, 341], [99, 25, 86], [200, 54, 176], [142, 14, 339], [18, 8, 222], [471, 28, 326], [25, 37, 82], [41, 38, 89], [447, 13, 339], [8, 51, 209], [62, 50, 108], [299, 29, 304], [

#### visualise resulting peaks using napari

#### compute peaks for a sample movie

In [91]:
sample_id = 0
movie = data[sample_id]
class_mask = annotations[sample_id]
event_mask = events[sample_id]

spark_mask = np.where(class_mask == 1, event_mask, 0)

In [92]:
spark_loc = detect_spark_peaks(movie=movie, instances_mask=spark_mask)

print("sparks locations:")
print(spark_loc)

sparks locations:
[[196, 41, 303], [186, 49, 142], [58, 25, 108], [64, 15, 107], [107, 10, 118], [77, 41, 102], [70, 46, 114], [79, 16, 194], [283, 20, 336], [354, 45, 163], [95, 56, 143], [371, 13, 255], [258, 28, 326], [312, 11, 113], [196, 22, 198], [445, 49, 75], [326, 20, 233], [211, 37, 82], [406, 54, 170], [10, 55, 122], [454, 40, 297], [262, 32, 291], [93, 11, 112], [38, 47, 67], [309, 28, 86], [141, 22, 197], [425, 51, 109], [101, 21, 305], [48, 44, 163], [25, 44, 245], [352, 44, 177], [405, 11, 362], [415, 20, 337], [65, 13, 190], [21, 8, 381], [228, 55, 170], [289, 16, 338], [82, 20, 255], [415, 28, 326], [69, 39, 297], [42, 25, 150], [303, 21, 138], [351, 52, 156], [58, 6, 351], [402, 53, 265], [133, 47, 182], [208, 56, 142], [475, 3, 344], [220, 13, 339], [124, 26, 85], [219, 30, 341], [99, 25, 86], [200, 54, 176], [142, 14, 339], [18, 8, 222], [471, 28, 326], [25, 37, 82], [41, 38, 89], [447, 13, 339], [8, 51, 209], [62, 50, 108], [299, 29, 304], [251, 3, 350], [43, 11, 1

In [93]:
viewer = napari.Viewer()
viewer.add_image(movie, name="movie", colormap=("colors", cmap))

viewer.add_labels(event_mask, name="events mask", opacity=0.5, visible=False)

viewer.add_labels(class_mask, name="classes mask", opacity=0.5, visible=False)

viewer.add_labels(spark_mask, name="sparks mask", opacity=0.6, visible=True)

viewer.add_points(spark_loc, name="spark peak locations", visible=True, size=3)

<Points layer 'spark peak locations' at 0x1c64388a7c0>